In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [3]:
from my_helper import *
from ML_lab_implement import *

In [5]:
std = np.std(tX, axis=0)
tX = tX / std

# required implementations
max_iters = 50
# least square
# gamma = 0.0000001
# logistic
gamma = 0.000001 
batch_size = 32
lambda_ = 0.1
# initial_w = np.random.rand(tX.shape[1]) * 2 - 1
initial_w = np.zeros(tX.shape[1])
# weights, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
# weights, loss = least_squares_SGD(y, tX, initial_w, batch_size, max_iters, gamma)
# weights, loss = least_squares(y, tX)
# weights, loss = ridge_regression(y, tX, lambda_)
# weights, loss = logistic_regression_GD(y, tX, initial_w, max_iters, gamma)
# weights, loss = reg_logistic_regression_SGD(y, tX, lambda_, initial_w, batch_size, max_iters, gamma)

In [11]:
weights

array([ 3.48383545e-02, -2.58363349e-01, -2.42364941e-01, -4.33964280e-02,
        1.13079342e-01,  3.94840833e-01, -1.10763277e-01,  2.45075194e-01,
        4.38642647e-03,  3.11424424e-02, -1.89968149e-01,  1.12977594e-01,
        9.73872335e-02,  1.73453941e-01, -8.91038812e-05, -8.39933650e-04,
        2.82106537e-01, -1.09432656e-03,  2.34801802e-03,  1.22388068e-01,
        1.00541880e-03, -7.20679795e-02, -3.19190244e-01, -2.77199398e-01,
        2.39466794e-01,  2.31879617e-01, -2.75879543e-01,  4.79854460e-02,
        2.98820086e-02, -6.63881069e-02])

In [10]:
# polynomial fit

ratio = 0.9
degree = 7
seed = 6

x_train, y_train, x_val, y_val = split_data(tX, y, ratio, seed)
x_train_poly = build_poly(x_train, degree)
x_val_poly = build_poly(x_val, degree)

lambdas = np.logspace(-5, 2, 30)

for lambda_ in lambdas:
    weights, loss = ridge_regression(y_train, x_train, lambda_)

    y_train_pred = predict_labels(weights, x_train)
    train_acc = cal_acc(y_train, y_train_pred)
    y_val_pred = predict_labels(weights, x_val)
    val_acc = cal_acc(y_val, y_val_pred)

    print("Lambda {labd}, Train accuracy {ta}, Validation Accuracy {va}".format(labd=lambda_, ta=train_acc, va=val_acc))

Lambda 1e-05, Train accuracy 0.7445111111111111, Validation Accuracy 0.74312
Lambda 1.7433288221999873e-05, Train accuracy 0.7445111111111111, Validation Accuracy 0.74316
Lambda 3.039195382313195e-05, Train accuracy 0.7445066666666666, Validation Accuracy 0.74312
Lambda 5.2983169062837125e-05, Train accuracy 0.7445111111111111, Validation Accuracy 0.74312
Lambda 9.236708571873866e-05, Train accuracy 0.7444933333333333, Validation Accuracy 0.74312
Lambda 0.0001610262027560939, Train accuracy 0.7444844444444444, Validation Accuracy 0.74304
Lambda 0.0002807216203941176, Train accuracy 0.7444755555555556, Validation Accuracy 0.74304
Lambda 0.0004893900918477494, Train accuracy 0.7444666666666667, Validation Accuracy 0.743
Lambda 0.0008531678524172815, Train accuracy 0.7444755555555556, Validation Accuracy 0.743
Lambda 0.0014873521072935117, Train accuracy 0.7444577777777778, Validation Accuracy 0.743
Lambda 0.002592943797404667, Train accuracy 0.7444755555555556, Validation Accuracy 0.7431

In [ ]:
# best para

ratio = 0.9
degree = 8
lambda_ = 5.2983169062837125e-05
seed = 6

x_poly = build_poly(tX, degree)
weights, loss = ridge_regression(y, x_poly, lambda_)

pred = predict_labels(weights, x_poly)
acc = cal_acc(y, pred)

print(acc)

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)